In [34]:
import numpy as np
import pandas as pd
from numpy import linalg
from numpy import random

data = pd.read_csv('datan.csv', sep=';').as_matrix()
datad = data.copy()
m = np.mean(data)
sdv = np.std(data)
data = (data - m)/sdv

#inicijalizacija matrice pripadnosti instance svakom klasteru
broj_instanci = data.shape[0]
broj_atributa = data.shape[1]
broj_klastera = np.random.randint(2,5)
p = np.zeros((broj_instanci,broj_klastera))

# inicijalizacija random gausovih parametra
cpl = []
for i in range(0, broj_klastera):
    cpl.append(np.random.uniform(0, 1))

cplp = cpl.copy()

cpm = []
for i in range(0, broj_klastera):
    tmp = np.zeros(broj_atributa)
    for j in range(0, broj_atributa):
        minv = data[:,j].min()
        maxv = data[:,j].max()
        tmp[j] = np.random.uniform(minv, maxv)
    cpm.append(tmp)
cpm = np.array(cpm)
    
cov = np.eye(broj_atributa)
cps = np.random.multivariate_normal(cpm[0], cov, (broj_atributa,broj_atributa))    
    
def verovatnoca(x, cpl, cpm, cps, broj_atributa):
    a = cpl*(1/((np.power((2*np.pi),broj_atributa/2)) * np.sqrt((np.linalg.norm(cps)))))
    b = np.exp((-np.dot(np.dot((x-cpm).T, cps),(x-cpm))/2))
    return a*b

def pripada_klasteru(p):
    ci = []
    for i in range(0, broj_instanci):
        a = np.array(p[i,:])
        i = np.where(a == a.max())
        ci.append(i[0][0])
    return ci

def normalize(value, suma):
    normalized = value/suma
    return normalized

def E_step(cpl, cpm, cps):
    p = np.zeros((broj_instanci, broj_klastera))
    for i in range(0, broj_instanci):
        for j in range(0, broj_klastera):
            p[i,j] = verovatnoca(data[i], cpl[j], cpm[j], cps[j], broj_atributa)

    pp = np.zeros((broj_instanci,broj_klastera))
    for i in range(0, broj_instanci):
        for j in range(0, broj_klastera):
            value = p[i,j]
            pp[i,j] = normalize(value, np.sum(p[i,:]))
    return pp;

def M_step(pp):
    #azuriranje cpl
    cplp = cpl.copy()
    for i in range(0, broj_klastera):
        cpl[i] = np.sum(pp[:,i])/broj_instanci

    #azuriranje cpm
    for i in range(0, broj_klastera):
        for j in range(0, broj_atributa):
            s = 0
            for k in range(0, broj_instanci):
                s += data[k,i] * pp[k, i]
            sc = np.sum(pp[:,i])
            cpm[i,j] = s/sc

    #azuriranje cps
    for i in range(0, broj_klastera):
        for j in range(0, broj_atributa):
            for k in range(0, broj_atributa):
                s = 0
                for l in range(0, broj_instanci):
                    pr = pp[l,i]
                    dr = data[l,:] - cpm[i]
                    tr = (data[l,:] - cpm[i]).reshape(broj_atributa, 1)
                    s += pr * np.dot(dr, tr)
                cps[i][j][k] = s/np.sum(pp[:,i])
    return cpl, cpm, cps, cplp


def konvergira(cpl, cplp, eps, iteracija, broj_atributa, br_iteracija):
    if iteracija == 1:
        return False
    
    if iteracija == br_iteracija:
        return True
    
    for i in range(0, broj_atributa):
        if np.abs(cpl[i] - cplp[i]) < eps:
            return True
        else:
            return False
    
    
eps = 0.01
iteracija = 1
br_iteracija = np.random.randint(10,50)

while(konvergira(cpl,cplp, eps, iteracija, broj_atributa, br_iteracija) != True):
    pp = E_step(cpl, cpm, cps)
    cpl, cpm, cps, cplp = M_step(pp)
    iteracija = iteracija + 1
    
pripada = np.array(pripada_klasteru(pp))
cluster = []
for i in range(0, broj_klastera):
    tmp = []
    for j in range(0, len(pripada)):
        if pripada[j] == i:
            tmp.append(datad[j,:])
    cluster.append(tmp)    

print('Broj potrebnih iteracija je:', iteracija-1, '\n')
for i in range(0, broj_klastera):
    print(i+1 , 'klasteru pripadaju sledece instance:')
    for j in range(0, len(cluster[i])):
        print(cluster[i][j])
    print('\n')
    

Broj potrebnih iteracija je: 32 

1 klasteru pripadaju sledece instance:
[ 6  0 66 50  1]
[ 2  1 70 50  2]
[ 5  0 69 50  3]
[ 4  0 68 50  4]
[ 3  0 67 50  5]
[ 6  0 72 50  6]
[  4   0  73 100   7]
[  3   0  70 100   8]
[ 3  1 70 50 22]


2 klasteru pripadaju sledece instance:
[  2   1  57 200   9]
[  6   1  63 200  10]
[  3   1  70 200  11]
[  4   0  78 200  12]
[  6   0  67 200  13]
[  6   2  53 200  14]
[  5   0  67 200  15]
[  6   0  75 200  16]
[  4   0  70 200  17]
[  6   0  81 200  18]
[  3   0  76 200  19]
[  6   0  79 200  20]
[  6   0  76 200  22]
[  2   1  58 200  23]


